In [27]:
from itertools import combinations,product,groupby
from collections import Counter
import numpy as np

In [245]:
class Frequent_items:
    def __init__(self, support):
        self.support = support
        with open("data/T10I4D100K.dat", "r") as f:
            self.itemsets = f.read().split("\n")
            self.itemsets = [set([int(x) for x in t.split()]) for t in self.itemsets]
        #self.itemsets = [{"A","C","D"}, {"B","C","E"}, {"A","B","C","E"}, {"B","E"}]
    def _count(self, sets):
        supports = {tuple(k):0 for k in sets}
        for t in self.itemsets:
            for c in sets:
                if c.issubset(t):
                    supports[tuple(c)] += 1
        return supports
    
    def get_ck(self, lprev):
        k = len(lprev[0])+1
        a = set()
        for elem1 in lprev:
            for elem2 in lprev:
                x = tuple(elem1 | elem2)
                if len(x) == k:
                    a.add(x)
        ck = []
        for elem in a:
            combs = list(combinations(elem,k-1))
            for i in range(len(combs)):
                if set(combs[i]) not in lprev:
                    break
                if i == len(combs)-1:
                    ck.append(set(elem))
        return self._count(ck)

    def get_Lk(self, cprev):
        return [set(k) for k,v in cprev.items() if v >= self.support]

In [ ]:
fi = Frequent_items(1000)
c1 = set()
for t in fi.itemsets:
    c1 = c1.union(t)
c1 = fi._count([{x} for x in c1])
l = fi.get_Lk(c1)
print(l)
answer = {tuple(set(x)) for x in l}
while l:
    ck = fi.get_ck(l)
    print("ck: " + str(Counter(ck).most_common(5)))
    l = fi.get_Lk(ck)
    print("l: " + str(l))
    answer = answer.union({tuple(set(x)) for x in l})
print(answer)

[{829}, {766}, {422}, {716}, {309}, {259}, {196}, {788}, {33}, {675}, {944}, {205}, {910}, {797}, {424}, {361}, {390}, {684}, {890}, {571}, {214}, {919}, {521}, {51}, {630}, {1}, {580}, {912}, {530}, {173}, {110}, {815}, {736}, {392}, {639}, {10}, {921}, {132}, {69}, {774}, {401}, {632}, {661}, {598}, {867}, {510}, {548}, {112}, {460}, {78}, {704}, {28}, {989}, {234}, {826}, {469}, {763}, {419}, {37}, {998}, {600}, {948}, {885}, {516}, {428}, {722}, {265}, {844}, {487}, {55}, {387}, {618}, {5}, {966}, {274}, {853}, {480}, {534}, {161}, {803}, {740}, {48}, {975}, {283}, {577}, {862}, {489}, {170}, {812}, {749}, {57}, {405}, {649}, {229}, {871}, {116}, {448}, {758}, {129}, {414}, {708}, {351}, {658}, {125}, {75}, {780}, {423}, {25}, {310}, {554}, {197}, {147}, {789}, {995}, {676}, {319}, {240}, {563}, {832}, {513}, {93}, {798}, {735}, {362}, {43}, {572}, {841}, {522}, {52}, {694}, {631}, {963}, {581}, {208}, {913}, {111}, {800}, {653}, {280}, {217}, {859}, {540}, {183}, {104}, {809}, {74